<!-- ![](https://europe-west1-atp-views-tracker.cloudfunctions.net/working-analytics?notebook=adaptive-retrieval) -->



# Adaptive Retrieval-Augmented Generation (RAG) System

## Overview

This system implements an advanced Retrieval-Augmented Generation (RAG) approach that adapts its retrieval strategy based on the type of query. By leveraging Language Models (LLMs) at various stages, it aims to provide more accurate, relevant, and context-aware responses to user queries.

## Motivation

Traditional RAG systems often use a one-size-fits-all approach to retrieval, which can be suboptimal for different types of queries. Our adaptive system is motivated by the understanding that different types of questions require different retrieval strategies. For example, a factual query might benefit from precise, focused retrieval, while an analytical query might require a broader, more diverse set of information.

## Key Components

1. **Query Classifier**: Determines the type of query (Factual, Analytical, Opinion, or Contextual).

2. **Adaptive Retrieval Strategies**: Four distinct strategies tailored to different query types:
   - Factual Strategy
   - Analytical Strategy
   - Opinion Strategy
   - Contextual Strategy

3. **LLM Integration**: LLMs are used throughout the process to enhance retrieval and ranking.

4. **OpenAI GPT Model**: Generates the final response using the retrieved documents as context.

## Method Details

### 1. Query Classification

The system begins by classifying the user's query into one of four categories:
- Factual: Queries seeking specific, verifiable information.
- Analytical: Queries requiring comprehensive analysis or explanation.
- Opinion: Queries about subjective matters or seeking diverse viewpoints.
- Contextual: Queries that depend on user-specific context.

### 2. Adaptive Retrieval Strategies

Each query type triggers a specific retrieval strategy:

#### Factual Strategy
- Enhances the original query using an LLM for better precision.
- Retrieves documents based on the enhanced query.
- Uses an LLM to rank documents by relevance.

#### Analytical Strategy
- Generates multiple sub-queries using an LLM to cover different aspects of the main query.
- Retrieves documents for each sub-query.
- Ensures diversity in the final document selection using an LLM.

#### Opinion Strategy
- Identifies different viewpoints on the topic using an LLM.
- Retrieves documents representing each viewpoint.
- Uses an LLM to select a diverse range of opinions from the retrieved documents.

#### Contextual Strategy
- Incorporates user-specific context into the query using an LLM.
- Performs retrieval based on the contextualized query.
- Ranks documents considering both relevance and user context.

### 3. LLM-Enhanced Ranking

After retrieval, each strategy uses an LLM to perform a final ranking of the documents. This step ensures that the most relevant and appropriate documents are selected for the next stage.

### 4. Response Generation

The final set of retrieved documents is passed to an OpenAI GPT model, which generates a response based on the query and the provided context.

## Benefits of This Approach

1. **Improved Accuracy**: By tailoring the retrieval strategy to the query type, the system can provide more accurate and relevant information.

2. **Flexibility**: The system adapts to different types of queries, handling a wide range of user needs.

3. **Context-Awareness**: Especially for contextual queries, the system can incorporate user-specific information for more personalized responses.

4. **Diverse Perspectives**: For opinion-based queries, the system actively seeks out and presents multiple viewpoints.

5. **Comprehensive Analysis**: The analytical strategy ensures a thorough exploration of complex topics.

## Conclusion

This adaptive RAG system represents a significant advancement over traditional RAG approaches. By dynamically adjusting its retrieval strategy and leveraging LLMs throughout the process, it aims to provide more accurate, relevant, and nuanced responses to a wide variety of user queries.

<div style="text-align: center;">

<img src="../images/adaptive_retrieval.svg" alt="adaptive retrieval" style="width:100%; height:auto;">
</div>

# Package Installation and Imports

The cell below installs all necessary packages required to run this notebook.


In [1]:
# Install required packages
!pip install faiss-cpu langchain langchain-openai python-dotenv


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [42]:
import os
import sys
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

from langchain_core.retrievers import BaseRetriever
from typing import Dict, Any, List
from langchain.docstore.document import Document
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain_core.pydantic_v1 import BaseModel, Field

### Define the query classifer class

In [3]:
class QueryClassifier:
    def __init__(self, model_name="llama3"):
        self.llm = OllamaLLM(model=model_name)
        self.prompt = PromptTemplate(
            input_variables=["query"],
            template=(
                "Classify the following query into one of these categories: "
                "Factual, Analytical, Opinion, or Contextual.\nQuery: {query}\nAnswer only with the category:"
            )
        )

    def classify(self, query: str) -> str:
        print("Classifying query...")
        formatted_prompt = self.prompt.format(query=query)
        raw_output = self.llm.invoke(formatted_prompt)

        # Extract only the first matching category
        for cat in ["Factual", "Analytical", "Opinion", "Contextual"]:
            if cat.lower() in raw_output.lower():
                return cat

        # Fallback in case LLM output is unexpected
        raise ValueError(f"Could not classify query. LLM output: {raw_output}")


### Define the Base Retriever class, such that the complex ones will inherit from it

In [4]:
class BaseRetrievalStrategy:
    def __init__(self, texts):
        self.embeddings = OllamaEmbeddings(model="nomic-embed-text")
        text_splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=0)
        self.documents = text_splitter.create_documents(texts)
        self.db = FAISS.from_documents(self.documents, self.embeddings)
        self.llm = OllamaLLM(model="llama3")


    def retrieve(self, query, k=4):
        return self.db.similarity_search(query, k=k)

### Define Factual retriever strategy

In [14]:
class RelevantScore(BaseModel):
    score: float = Field(description="The relevance score of the document to the query", example=8.0)

class FactualRetrievalStrategy(BaseRetrievalStrategy):
    def __init__(self, db, model_name="llama3"):
        self.db = db
        self.llm = OllamaLLM(model=model_name)

    def retrieve(self, query: str, k: int = 4) -> List:
        print("retrieving factual")

        # Step 1: Enhance the query using the LLM
        enhanced_query_prompt = PromptTemplate(
            input_variables=["query"],
            template="Enhance this factual query for better information retrieval: {query}"
        )
        enhanced_query = self.llm.invoke(enhanced_query_prompt.format(query=query))
        print(f"Enhanced query: {enhanced_query}")

        # Step 2: Retrieve documents using the enhanced query
        docs = self.db.similarity_search(enhanced_query, k=k*2)

        # Step 3: Rank documents using the LLM
        ranking_prompt = PromptTemplate(
            input_variables=["query", "doc"],
            template="On a scale of 1-10, how relevant is this document to the query: '{query}'?\nDocument: {doc}\nRelevance score:"
        )

        ranked_docs = []
        print("ranking docs")
        for doc in docs:
            input_text = ranking_prompt.format(query=enhanced_query, doc=doc.page_content)
            score_output = self.llm.invoke(input_text, output_class=RelevantScore)
            score = float(score_output.score)
            ranked_docs.append((doc, score))

        # Step 4: Sort by relevance and return top k
        ranked_docs.sort(key=lambda x: x[1], reverse=True)
        return [doc for doc, _ in ranked_docs[:k]]


### Define Analytical retriever strategy

In [15]:
class SelectedIndices(BaseModel):
    indices: List[int] = Field(description="Indices of selected documents", example=[0, 1, 2, 3])

class SubQueries(BaseModel):
    sub_queries: List[str] = Field(description="List of sub-queries for comprehensive analysis", 
                                   example=["What is the population of New York?", "What is the GDP of New York?"])

class AnalyticalRetrievalStrategy(BaseRetrievalStrategy):
    def __init__(self, db, model_name="llama3"):
        self.db = db
        self.llm = OllamaLLM(model=model_name)

    def retrieve(self, query: str, k: int = 4) -> List:
        print("retrieving analytical")

        # Step 1: Generate sub-queries for comprehensive analysis
        sub_queries_prompt = PromptTemplate(
            input_variables=["query", "k"],
            template="Generate {k} sub-questions for: {query}"
        )
        sub_queries_output = self.llm.invoke(sub_queries_prompt.format(query=query, k=k), output_class=SubQueries)
        sub_queries = sub_queries_output.sub_queries
        print(f'Sub-queries for comprehensive analysis: {sub_queries}')

        # Step 2: Retrieve documents for each sub-query
        all_docs = []
        for sub_query in sub_queries:
            all_docs.extend(self.db.similarity_search(sub_query, k=2))

        # Step 3: Select diverse and relevant documents
        diversity_prompt = PromptTemplate(
            input_variables=["query", "docs", "k"],
            template="""Select the most diverse and relevant set of {k} documents for the query: '{query}'.
Return only the indices of selected documents as a list of integers.
Documents: {docs}"""
        )

        docs_text = "\n".join([f"{i}: {doc.page_content[:50]}..." for i, doc in enumerate(all_docs)])
        selected_output = self.llm.invoke(
            diversity_prompt.format(query=query, docs=docs_text, k=k),
            output_class=SelectedIndices
        )
        selected_indices = [i for i in selected_output.indices if i < len(all_docs)]
        print(f'Selected diverse and relevant documents: {selected_indices}')

        return [all_docs[i] for i in selected_indices]


### Define Opinion retriever strategy

In [16]:
# Define structured output for selected indices
class SelectedIndices(BaseModel):
    indices: str = Field(description="Space-separated indices of selected documents")

class OpinionRetrievalStrategy(BaseRetrievalStrategy):
    def __init__(self, db, model_name="llama3"):
        """
        db: vector database
        model_name: Ollama LLM model name
        """
        self.db = db
        self.llm = OllamaLLM(model=model_name)

    def retrieve(self, query: str, k: int = 3) -> List:
        print("retrieving opinion")

        # Step 1: Generate viewpoints using Ollama LLM
        viewpoints_prompt = PromptTemplate(
            input_variables=["query", "k"],
            template="Identify {k} distinct viewpoints or perspectives on the topic: {query}"
        )
        input_data = {"query": query, "k": k}
        viewpoints_text = self.llm.invoke(viewpoints_prompt.format(**input_data)).content
        viewpoints = [vp.strip() for vp in viewpoints_text.split('\n') if vp.strip()]
        print(f'viewpoints: {viewpoints}')

        # Step 2: Retrieve documents for each viewpoint
        all_docs = []
        for viewpoint in viewpoints:
            all_docs.extend(self.db.similarity_search(f"{query} {viewpoint}", k=2))

        # Step 3: Classify and select diverse opinions
        opinion_prompt = PromptTemplate(
            input_variables=["query", "docs", "k"],
            template=(
                "Classify these documents into distinct opinions on '{query}' "
                "and select the {k} most representative and diverse viewpoints:\n"
                "Documents: {docs}\nSelected indices:"
            )
        )

        docs_text = "\n".join([f"{i}: {doc.page_content[:100]}..." for i, doc in enumerate(all_docs)])
        input_data = {"query": query, "docs": docs_text, "k": k}

        # Use Ollama LLM with structured output
        selected_output = self.llm.invoke(opinion_prompt.format(**input_data), output_class=SelectedIndices)
        selected_indices = [int(i) for i in selected_output.indices.split() if i.isdigit() and int(i) < len(all_docs)]

        print(f'selected diverse and relevant documents: {selected_indices}')

        return [all_docs[i] for i in selected_indices]


### Define Contextual retriever strategy

In [17]:

class RelevanceScore(BaseModel):
    score: float = Field(description="Relevance score of the document from 1 to 10")

class ContextualRetrievalStrategy(BaseRetrievalStrategy):
    def __init__(self, db, model_name="llama3"):
        """
        db: vector database
        model_name: Ollama model name
        """
        self.db = db
        self.llm = OllamaLLM(model=model_name)

    def retrieve(self, query: str, k: int = 4, user_context: str = None) -> List:
        print("retrieving contextual")

        # Step 1: Contextualize the query using Ollama LLM
        context_prompt = PromptTemplate(
            input_variables=["query", "context"],
            template="Given the user context: {context}\nReformulate the query to best address the user's needs: {query}"
        )
        input_data = {"query": query, "context": user_context or "No specific context provided"}
        contextualized_query = self.llm.invoke(context_prompt.format(**input_data)).content
        print(f'contextualized query: {contextualized_query}')

        # Step 2: Retrieve documents using the contextualized query
        docs = self.db.similarity_search(contextualized_query, k=k*2)

        # Step 3: Rank documents using Ollama LLM with structured output
        ranking_prompt = PromptTemplate(
            input_variables=["query", "context", "doc"],
            template="Given the query: '{query}' and user context: '{context}', rate the relevance of this document on a scale of 1-10:\nDocument: {doc}\nRelevance score:"
        )

        ranked_docs = []
        for doc in docs:
            input_data = {
                "query": contextualized_query,
                "context": user_context or "No specific context provided",
                "doc": doc.page_content
            }
            # Use Ollama LLM with Pydantic schema for structured output
            score_output = self.llm.invoke(ranking_prompt.format(**input_data), output_class=RelevanceScore)
            ranked_docs.append((doc, score_output.score))

        # Step 4: Sort by relevance score and return top-k
        ranked_docs.sort(key=lambda x: x[1], reverse=True)
        return [doc for doc, _ in ranked_docs[:k]]


### Define the Adapive retriever class

In [18]:
class AdaptiveRetriever:
    def __init__(self, texts: List[str]):
        self.classifier = QueryClassifier()
        self.strategies = {
            "Factual": FactualRetrievalStrategy(texts),
            "Analytical": AnalyticalRetrievalStrategy(texts),
            "Opinion": OpinionRetrievalStrategy(texts),
            "Contextual": ContextualRetrievalStrategy(texts)
        }

    def get_relevant_documents(self, query: str) -> List[Document]:
        category = self.classifier.classify(query)
        strategy = self.strategies[category]
        return strategy.retrieve(query)

### Define aditional retriever that inherits from langchain BaseRetriever 

In [19]:
class PydanticAdaptiveRetriever(BaseRetriever):
    adaptive_retriever: AdaptiveRetriever = Field(exclude=True)

    class Config:
        arbitrary_types_allowed = True

    def get_relevant_documents(self, query: str) -> List[Document]:
        return self.adaptive_retriever.get_relevant_documents(query)

    async def aget_relevant_documents(self, query: str) -> List[Document]:
        return self.get_relevant_documents(query)

/var/folders/bx/b60shh3d5fgcy2bhfzqsdnn00000gn/T/ipykernel_9703/4016186069.py:1: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class PydanticAdaptiveRetriever(BaseRetriever):
/var/folders/bx/b60shh3d5fgcy2bhfzqsdnn00000gn/T/ipykernel_9703/4016186069.py:1: DeprecationWarning: Retrievers must implement abstract `_aget_relevant_documents` method instead of `aget_relevant_documents`
  class PydanticAdaptiveRetriever(BaseRetriever):


### Define the Adaptive RAG class

In [27]:
def replace_t_with_space(list_of_documents):
    """
    Replaces all tab characters ('\t') with spaces in the page content of each document

    Args:
        list_of_documents: A list of document objects, each with a 'page_content' attribute.

    Returns:
        The modified list of documents with tab characters replaced by spaces.
    """

    for doc in list_of_documents:
        doc.page_content = doc.page_content.replace('\t', ' ')  # Replace tabs with spaces
    return list_of_documents

In [36]:
class AdaptiveRAG:
    def __init__(self, texts: List[str]):
        adaptive_retriever = AdaptiveRetriever(texts)
        self.retriever = PydanticAdaptiveRetriever(adaptive_retriever=adaptive_retriever)
        self.llm = OllamaLLM(model="llama3")
        
        # Create a custom prompt
        prompt_template = """Use the following pieces of context to answer the question at the end. 
        If you don't know the answer, just say that you don't know, don't try to make up an answer.

        {context}

        Question: {question}
        Answer:"""
        prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
        
        # Create the LLM chain
        self.llm_chain = prompt | self.llm
        
      

    def answer(self, query: str) -> str:
        docs = self.retriever.get_relevant_documents(query)
        input_data = {"context": "\n".join([doc.page_content for doc in docs]), "question": query}
        return self.llm_chain.invoke(input_data)

### Demonstrate use of this model

In [40]:
# Usage
texts = [
    "The Earth is the third planet from the Sun and the only astronomical object known to harbor life."
    ]
rag_system = AdaptiveRAG(texts)

### Showcase the four different types of queries

In [41]:
factual_result = rag_system.answer("What is the distance between the Earth and the Sun?").content
print(f"Answer: {factual_result}")

analytical_result = rag_system.answer("How does the Earth's distance from the Sun affect its climate?").content
print(f"Answer: {analytical_result}")

opinion_result = rag_system.answer("What are the different theories about the origin of life on Earth?").content
print(f"Answer: {opinion_result}")

contextual_result = rag_system.answer("How does the Earth's position in the Solar System influence its habitability?").content
print(f"Answer: {contextual_result}")

Classifying query...
retrieving factual
Enhanced query: To enhance this factual query, we can make it more specific and detailed to retrieve more accurate and relevant information. Here are some suggestions:

1. **Specificity**: Instead of asking about the "distance" between the two bodies, you could ask about the **average distance**, which is a well-defined value (about 149.6 million kilometers or 92.96 million miles).
2. **Unit**: Specify the unit of measurement you're interested in. For example: "What is the average distance between Earth and Sun in kilometers?" or "What is the average distance between Earth and Sun in astronomical units?"
3. **Context**: Consider adding context to your query to help narrow down the results. For instance, you could ask about the:
	* Perihelion (closest point) or aphelion (farthest point) of the Earth's orbit around the Sun.
	* Distance at a specific time or date (e.g., "What is the distance between Earth and Sun on June 21st?")
4. **Additional info

AttributeError: 'list' object has no attribute 'similarity_search'

![](https://europe-west1-rag-techniques-views-tracker.cloudfunctions.net/rag-techniques-tracker?notebook=all-rag-techniques--adaptive-retrieval)

In [ ]:
import os
from typing import List
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain_ollama import OllamaLLM, OllamaEmbeddings

# ----------------------------
# Query Classifier
# ----------------------------
class QueryClassifier:
    def __init__(self, model_name="llama3"):
        self.llm = OllamaLLM(model=model_name)
        self.prompt = PromptTemplate(
            input_variables=["query"],
            template=(
                "Classify the following query into one of these categories: "
                "Factual, Analytical, Opinion, or Contextual.\nQuery: {query}\nAnswer only with the category:"
            )
        )

    def classify(self, query: str) -> str:
        print("Classifying query...")
        formatted_prompt = self.prompt.format(query=query)
        raw_output = self.llm.invoke(formatted_prompt)
        # Extract only the first matching category
        for cat in ["Factual", "Analytical", "Opinion", "Contextual"]:
            if cat.lower() in raw_output.lower():
                return cat
        raise ValueError(f"Could not classify query. LLM output: {raw_output}")

# ----------------------------
# Base Retrieval Strategy
# ----------------------------
class BaseRetrievalStrategy:
    def __init__(self, texts: List[str]):
        self.embeddings = OllamaEmbeddings(model="nomic-embed-text")
        text_splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=0)
        self.documents = text_splitter.create_documents(texts)
        self.db = FAISS.from_documents(self.documents, self.embeddings)
        self.llm = OllamaLLM(model="llama3")

    def retrieve(self, query: str, k: int = 4) -> List[Document]:
        return self.db.similarity_search(query, k=k)

# ----------------------------
# Factual Strategy
# ----------------------------
class FactualRetrievalStrategy(BaseRetrievalStrategy):
    def retrieve(self, query: str, k: int = 4) -> List[Document]:
        print("Retrieving factual documents...")
        # Step 1: Enhance query
        prompt = f"Enhance this factual query for better information retrieval: {query}"
        enhanced_query = self.llm.invoke(prompt).strip()
        print(f"Enhanced query: {enhanced_query}")

        # Step 2: Retrieve
        docs = self.db.similarity_search(enhanced_query, k=k*2)

        # Step 3: Rank documents
        ranking_prompt_template = "On a scale of 1-10, how relevant is this document to the query: '{query}'?\nDocument: {doc}\nRelevance score:"
        ranked_docs = []
        for doc in docs:
            input_text = ranking_prompt_template.format(query=enhanced_query, doc=doc.page_content)
            score_str = self.llm.invoke(input_text).strip()
            try:
                score = float(score_str)
            except ValueError:
                score = 0.0
            ranked_docs.append((doc, score))

        # Step 4: Return top-k
        ranked_docs.sort(key=lambda x: x[1], reverse=True)
        return [doc for doc, _ in ranked_docs[:k]]

# ----------------------------
# Analytical Strategy
# ----------------------------
class AnalyticalRetrievalStrategy(BaseRetrievalStrategy):
    def retrieve(self, query: str, k: int = 4) -> List[Document]:
        print("Retrieving analytical documents...")
        # Step 1: Generate sub-queries
        prompt = f"Generate {k} sub-questions for: {query}"
        sub_queries_text = self.llm.invoke(prompt).strip()
        sub_queries = [sq.strip() for sq in sub_queries_text.split("\n") if sq.strip()]
        print(f"Sub-queries: {sub_queries}")

        # Step 2: Retrieve for each sub-query
        all_docs = []
        for sq in sub_queries:
            all_docs.extend(self.db.similarity_search(sq, k=2))

        # Step 3: Select top k manually (just take first k for simplicity)
        return all_docs[:k]

# ----------------------------
# Opinion Strategy
# ----------------------------
class OpinionRetrievalStrategy(BaseRetrievalStrategy):
    def retrieve(self, query: str, k: int = 3) -> List[Document]:
        print("Retrieving opinion documents...")
        # Step 1: Generate viewpoints
        prompt = f"Identify {k} distinct viewpoints or perspectives on the topic: {query}"
        viewpoints_text = self.llm.invoke(prompt).strip()
        viewpoints = [vp.strip() for vp in viewpoints_text.split("\n") if vp.strip()]
        print(f"Viewpoints: {viewpoints}")

        # Step 2: Retrieve for each viewpoint
        all_docs = []
        for vp in viewpoints:
            all_docs.extend(self.db.similarity_search(f"{query} {vp}", k=2))

        # Step 3: Return first k
        return all_docs[:k]

# ----------------------------
# Contextual Strategy
# ----------------------------
class ContextualRetrievalStrategy(BaseRetrievalStrategy):
    def retrieve(self, query: str, k: int = 4, user_context: str = None) -> List[Document]:
        print("Retrieving contextual documents...")
        context_prompt = f"Given the user context: {user_context or 'No specific context provided'}\nReformulate the query: {query}"
        contextual_query = self.llm.invoke(context_prompt).strip()
        print(f"Contextualized query: {contextual_query}")

        docs = self.db.similarity_search(contextual_query, k=k*2)
        return docs[:k]

# ----------------------------
# Adaptive Retriever
# ----------------------------
class AdaptiveRetriever:
    def __init__(self, texts: List[str]):
        self.classifier = QueryClassifier()
        self.strategies = {
            "Factual": FactualRetrievalStrategy(texts),
            "Analytical": AnalyticalRetrievalStrategy(texts),
            "Opinion": OpinionRetrievalStrategy(texts),
            "Contextual": ContextualRetrievalStrategy(texts)
        }

    def get_relevant_documents(self, query: str) -> List[Document]:
        category = self.classifier.classify(query)
        print(f"Query category: {category}")
        strategy = self.strategies[category]
        return strategy.retrieve(query)

# ----------------------------
# Adaptive RAG System
# ----------------------------
class AdaptiveRAG:
    def __init__(self, texts: List[str]):
        self.retriever = AdaptiveRetriever(texts)
        self.llm = OllamaLLM(model="llama3")
        prompt_template = """Use the following context to answer the question.
{context}

Question: {question}
Answer:"""
        self.prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    def answer(self, query: str) -> str:
        docs = self.retriever.get_relevant_documents(query)
        context_text = "\n".join([doc.page_content for doc in docs])
        input_data = self.prompt.format(context=context_text, question=query)
        return self.llm.invoke(input_data).strip()

# ----------------------------
# Usage Example
# ----------------------------
texts = [
    "The Earth is the third planet from the Sun and the only astronomical object known to harbor life."
]

rag_system = AdaptiveRAG(texts)

# Factual
factual_result = rag_system.answer("What is the distance between the Earth and the Sun?")
print("Factual answer:", factual_result)

# Analytical
analytical_result = rag_system.answer("How does the Earth's distance from the Sun affect its climate?")
print("Analytical answer:", analytical_result)

# Opinion
opinion_result = rag_system.answer("What are the different theories about the origin of life on Earth?")
print("Opinion answer:", opinion_result)

# Contextual
contextual_result = rag_system.answer("How does the Earth's position in the Solar System influence its habitability?")
print("Contextual answer:", contextual_result)


Classifying query...
Query category: Factual
Retrieving factual documents...
